# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 4453, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 4453 (delta 29), reused 17 (delta 6), pack-reused 4388 (from 1)
Receiving objects: 100% (4453/4453), 174.83 MiB | 31.13 MiB/s, done.
Resolving deltas: 100% (2610/2610), done.
Updating files: 100% (406/406), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.6 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationI

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV/SLIM'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SLIMElasticNet',
    'metric': 'Recall',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIMElasticNet_Recall.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[50])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [12]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

def objective_function_SLIMElasticNet(optuna_trial):
    
    recommender_instance = SLIMElasticNetRecommender(URM_train)
    
    full_hyperp = {
                   "topK": optuna_trial.suggest_int("topK", 000, 1500),
                   "l1_ratio": optuna_trial.suggest_float("l1_ratio", 0.01, 1.0, log=True),
                    "alpha": optuna_trial.suggest_float("alpha", 1e-4, 1e-1, log=True),
                    "positive_only": optuna_trial.suggest_categorical("positive_only", [True, False]),
                  }        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[50]["RECALL"]

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_SLIMElasticNet, n_trials=5)

[I 2024-12-24 16:38:07,922] Using an existing study with name 'hyperparameters_tuning_SLIMElasticNet_Recall' instead of creating a new one.


SLIMElasticNetRecommender: Processed 2842 ( 7.5%) in 5.00 min. Items per second: 9.47
SLIMElasticNetRecommender: Processed 5707 (15.0%) in 10.00 min. Items per second: 9.51
SLIMElasticNetRecommender: Processed 8587 (22.5%) in 15.00 min. Items per second: 9.54
SLIMElasticNetRecommender: Processed 11483 (30.1%) in 20.00 min. Items per second: 9.57
SLIMElasticNetRecommender: Processed 14309 (37.5%) in 25.00 min. Items per second: 9.54
SLIMElasticNetRecommender: Processed 17256 (45.3%) in 30.00 min. Items per second: 9.58
SLIMElasticNetRecommender: Processed 20264 (53.2%) in 35.01 min. Items per second: 9.65
SLIMElasticNetRecommender: Processed 23165 (60.8%) in 40.01 min. Items per second: 9.65
SLIMElasticNetRecommender: Processed 26177 (68.7%) in 45.01 min. Items per second: 9.69
SLIMElasticNetRecommender: Processed 29216 (76.6%) in 50.01 min. Items per second: 9.74
SLIMElasticNetRecommender: Processed 32234 (84.6%) in 55.01 min. Items per second: 9.77
SLIMElasticNetRecommender: Processed

[I 2024-12-24 17:44:01,982] Trial 27 finished with value: 0.282733188452228 and parameters: {'topK': 938, 'l1_ratio': 0.01627118613873808, 'alpha': 0.0017369858435446462, 'positive_only': True}. Best is trial 27 with value: 0.282733188452228.


SLIMElasticNetRecommender: Processed 2728 ( 7.2%) in 5.00 min. Items per second: 9.09
SLIMElasticNetRecommender: Processed 5582 (14.6%) in 10.00 min. Items per second: 9.30
SLIMElasticNetRecommender: Processed 8458 (22.2%) in 15.00 min. Items per second: 9.39
SLIMElasticNetRecommender: Processed 11211 (29.4%) in 20.00 min. Items per second: 9.34
SLIMElasticNetRecommender: Processed 14011 (36.8%) in 25.00 min. Items per second: 9.34
SLIMElasticNetRecommender: Processed 16958 (44.5%) in 30.01 min. Items per second: 9.42
SLIMElasticNetRecommender: Processed 20044 (52.6%) in 35.01 min. Items per second: 9.54
SLIMElasticNetRecommender: Processed 23099 (60.6%) in 40.01 min. Items per second: 9.62
SLIMElasticNetRecommender: Processed 26171 (68.7%) in 45.01 min. Items per second: 9.69
SLIMElasticNetRecommender: Processed 29107 (76.4%) in 50.01 min. Items per second: 9.70
SLIMElasticNetRecommender: Processed 32036 (84.0%) in 55.01 min. Items per second: 9.71
SLIMElasticNetRecommender: Processed

[I 2024-12-24 18:49:58,344] Trial 28 finished with value: 0.2822423120387647 and parameters: {'topK': 949, 'l1_ratio': 0.014781556291321834, 'alpha': 0.0019665100362091627, 'positive_only': True}. Best is trial 27 with value: 0.282733188452228.


...

[I 2024-12-24 19:54:33,161] Trial 29 finished with value: 0.2807683336548563 and parameters: {'topK': 940, 'l1_ratio': 0.016089554601649632, 'alpha': 0.002179946234023015, 'positive_only': True}. Best is trial 27 with value: 0.282733188452228.


SLIMElasticNetRecommender: Processed 3108 ( 8.2%) in 5.00 min. Items per second: 10.36
SLIMElasticNetRecommender: Processed 6081 (16.0%) in 10.00 min. Items per second: 10.13
SLIMElasticNetRecommender: Processed 9227 (24.2%) in 15.00 min. Items per second: 10.25
SLIMElasticNetRecommender: Processed 12385 (32.5%) in 20.00 min. Items per second: 10.32
SLIMElasticNetRecommender: Processed 15654 (41.1%) in 25.00 min. Items per second: 10.43
SLIMElasticNetRecommender: Processed 18761 (49.2%) in 30.00 min. Items per second: 10.42
SLIMElasticNetRecommender: Processed 21977 (57.7%) in 35.00 min. Items per second: 10.46
SLIMElasticNetRecommender: Processed 25145 (66.0%) in 40.00 min. Items per second: 10.48
SLIMElasticNetRecommender: Processed 28362 (74.4%) in 45.00 min. Items per second: 10.50
SLIMElasticNetRecommender: Processed 31607 (82.9%) in 50.01 min. Items per second: 10.53
SLIMElasticNetRecommender: Processed 34988 (91.8%) in 55.01 min. Items per second: 10.60
SLIMElasticNetRecommender

[I 2024-12-24 20:55:12,875] Trial 30 finished with value: 0.273703267342652 and parameters: {'topK': 1218, 'l1_ratio': 0.016214768505100927, 'alpha': 0.003180566690827202, 'positive_only': True}. Best is trial 27 with value: 0.282733188452228.


SLIMElasticNetRecommender: Processed 2835 ( 7.4%) in 5.00 min. Items per second: 9.44
SLIMElasticNetRecommender: Processed 5554 (14.6%) in 10.00 min. Items per second: 9.25
SLIMElasticNetRecommender: Processed 8594 (22.5%) in 15.00 min. Items per second: 9.55
SLIMElasticNetRecommender: Processed 11527 (30.2%) in 20.00 min. Items per second: 9.60
SLIMElasticNetRecommender: Processed 14613 (38.3%) in 25.00 min. Items per second: 9.74
SLIMElasticNetRecommender: Processed 17583 (46.1%) in 30.00 min. Items per second: 9.77
SLIMElasticNetRecommender: Processed 20605 (54.1%) in 35.00 min. Items per second: 9.81
SLIMElasticNetRecommender: Processed 23480 (61.6%) in 40.01 min. Items per second: 9.78
SLIMElasticNetRecommender: Processed 26560 (69.7%) in 45.01 min. Items per second: 9.84
SLIMElasticNetRecommender: Processed 29487 (77.4%) in 50.01 min. Items per second: 9.83
SLIMElasticNetRecommender: Processed 32456 (85.1%) in 55.01 min. Items per second: 9.83
SLIMElasticNetRecommender: Processed

[I 2024-12-24 22:00:17,412] Trial 31 finished with value: 0.28060383717283277 and parameters: {'topK': 870, 'l1_ratio': 0.026395479851558903, 'alpha': 0.0015897512428938384, 'positive_only': True}. Best is trial 27 with value: 0.282733188452228.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = SLIMElasticNetRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

SLIMElasticNetRecommender: Processed 2153 ( 5.6%) in 5.00 min. Items per second: 7.17
SLIMElasticNetRecommender: Processed 4361 (11.4%) in 10.00 min. Items per second: 7.27
SLIMElasticNetRecommender: Processed 6616 (17.4%) in 15.00 min. Items per second: 7.35
SLIMElasticNetRecommender: Processed 8913 (23.4%) in 20.00 min. Items per second: 7.43
SLIMElasticNetRecommender: Processed 11223 (29.4%) in 25.00 min. Items per second: 7.48
SLIMElasticNetRecommender: Processed 13533 (35.5%) in 30.01 min. Items per second: 7.52
SLIMElasticNetRecommender: Processed 15808 (41.5%) in 35.01 min. Items per second: 7.53
SLIMElasticNetRecommender: Processed 17997 (47.2%) in 40.01 min. Items per second: 7.50
SLIMElasticNetRecommender: Processed 20179 (52.9%) in 45.01 min. Items per second: 7.47
SLIMElasticNetRecommender: Processed 22368 (58.7%) in 50.01 min. Items per second: 7.45
SLIMElasticNetRecommender: Processed 24624 (64.6%) in 55.01 min. Items per second: 7.46
SLIMElasticNetRecommender: Processed 

# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_SLIMElasticNet_Recall.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithoutKFCV/SLIM/SLIMElasticNetRecommender/OptimizingRecall/best_params_SLIMElasticNet_Recall.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithoutKFCV/SLIM/SLIMElasticNetRecommender/OptimizingRecall/history_SLIMElasticNet_Recall.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithoutKFCV/SLIM/SLIMElasticNetRecommender/OptimizingRecall/Submission/submission_SLIMElasticNet_Recall.csv' updated successfully.
